# Web Scraping with Python and Selenium

## Libraries and settings

In [ ]:
# Libraries
import os
import re
import json
import time
import random
import pandas as pd

from bs4 import BeautifulSoup

from prettytable import from_csv

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

# Set up Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Path to ChromeDriver
service = Service('/usr/local/bin/chromedriver')
# service = Service(executable_path=r'C:\Tools\chromedriver\chromedriver.exe')

# Settings
import warnings
warnings.filterwarnings("ignore")

# Current working directory
print(f'Current working directory: {os.getcwd()}')

## Import a list with user agents

In [ ]:
# Liste mit User-Agents für Rotation
ua_path = "user_agents.txt"
ua_list = [line.rstrip('\n') for line in open(ua_path)]
ua_list[:5]

## Chrome headless mode (without graphical user interface)

In [ ]:
# Initialize the driver
driver = webdriver.Chrome(service=service, options=options)

# Website to scrape
driver.get('https://store.dji.com')

# Get current url
print(driver.current_url)

# Return the full page HTML code
# print(driver.page_source)

# Gets the page's title
print(driver.title)

# Close driver
driver.close()


## Creating screenshots from a website

In [ ]:
# Chrome-Driver
driver = webdriver.Chrome(service=service, options=options)

# Website to scrape
driver.get('https://store.dji.com')

# Screenshot
driver.save_screenshot('screenshot.png')

# Plot image
img = mpimg.imread('screenshot.png')
plt.figure(figsize=(8,8))
plt.axis('off')
imgplot = plt.imshow(img)

# Close driver
driver.close()

## Locating elements

There are different methods available in the Selenium API to select elements on the page. You can use:

- Name
- Tag name
- Class name
- ID
- XPath
- CSS selectors

Use the inspect element function in Chrome to get these infos:
https://www.hostinger.com/tutorials/website/how-to-inspect-and-change-style-using-google-chrome

### Locating elements using XPath

In [ ]:
# Chrome-Driver
driver = webdriver.Chrome(service=service, options=options)

# Website to scrape
driver.get('https://store.dji.com')

# Find element using xpath
text = driver.find_element(By.XPATH, '/html/body').text
print(text[:50])

# Close driver
driver.close()

### Locating elements on https://www.autoscout24.ch

In [ ]:
# Change options to regular mode
options = webdriver.ChromeOptions()

# Chrome-Driver
driver = webdriver.Chrome(service=service, options=options)

# Website to scrape
url = 'https://www.autoscout24.ch/de/s/mk-vw?pagination%5Bpage%5D=1'
driver.get(url)

# Get webpage number from url
pattern = r'=(.*)'
page = re.search(pattern, url)

# Get the full content of the website
source = driver.page_source

# Close driver
driver.close()

# Parse HTML content with BeautifulSoup
soup = BeautifulSoup(source, 'html.parser')

# Get links
links = []
link_elements = soup.find_all(class_='css-qqdlpa')
for element in link_elements:
    links.append('https://www.autoscout24.ch' + element['href'])

# Get types
types_elements = soup.find_all(class_='chakra-heading css-svmyrf')
types = [element.text for element in types_elements]

# Get prices
prices_elements = soup.find_all(class_='chakra-text css-bwl0or')
prices = [element.text for element in prices_elements]

# Dataframe
df = pd.DataFrame({ 'Page': page.group(1),
                    'Type': types,
                    'Price': prices,
                    'Weblink': links})

# Save to file
df.to_csv('autoscout24.csv', sep=";", index=False)

# Show file
with open('autoscout24.csv') as table_file:
    tab = from_csv(table_file, delimiter=';')
    
    # Set the alignment for all columns to 'l' (left-align)
    for field_name in tab.field_names:
        tab.align[field_name] = 'l'
    
    print(tab)

### Locating elements using tag names

In [ ]:
# Chrome-Driver
driver = webdriver.Chrome(service=service, options=options)

# Website to scrape
driver.get('https://bienen.ch')

# Find element using tag names
text = driver.find_element(By.TAG_NAME, 'h3').text
print(text)

# Close driver
driver.close()

## Sent and submit text on Wikipedia

In [ ]:
# Chrome-Driver
driver = webdriver.Chrome(service=service, options=options)

# Website to scrape
driver.get('https://de.wikipedia.org')

# Fill in the search text
text_to_send = 'Currywurst'
element = driver.find_element(By.XPATH, '/html/body/div[4]/div[1]/div[2]/div/form/div/input[1]')
element.send_keys(text_to_send)

# Submit search text
element.submit()

# Extract text
element = driver.find_element(By.TAG_NAME, 'p').text
print(element)

# Wait until the website is showing up
time.sleep(5)

# Click link to image on page found
driver.find_element(By.XPATH, '/html/body/div[3]/div[3]/div[5]/div[1]/figure[1]/a/img').click()

# Screenshot
time.sleep(5)
driver.save_screenshot('screenshot.png')

# Close driver
driver.close()

# Plot image
img = mpimg.imread('screenshot.png')
plt.figure(figsize=(6,6))
plt.axis('off')
imgplot = plt.imshow(img)

## Pagination handling on https://www.minergie.ch

In [ ]:
# Chrome-Driver
driver = webdriver.Chrome(service=service, options=options)

# Open file-connection
MyFile = open('minergie_objects.txt', 'w', encoding='utf-8')

obj = {}    

# Loop through first 5 pages and save objects to file
for i in range(5):
    
    # Generating URLs using page numbers
    url = str('https://www.minergie.ch/de/gebaeude/gebaeudeliste/?canton=&country=&zip_place=&street_nr=&gid=&participator=&typeofuse=&constructiontype=&year=&sortby=date_asc&numres=12&p=' + str(i+1))
    time.sleep(1)
    driver.get(url)
    
    # Click 'I agree' button for cookie-management on first page
    if i == 1:
        driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div[3]/div[1]/button[3]').click()
    else:
        pass
    
    print(str('--------- Page:' + str(i+1) + '-------------------------------'))
    
    for n in range(12):
        
        path = str('/html/body/div[1]/div[4]/div[7]/div/div[' + str(n+1) + ']')

        obj['Object'] = driver.find_element(By.XPATH, path).text
        obj['Object'] = obj['Object'].replace("\n", "")
        print(obj['Object'])
        
        # Write to file
        MyFile.write(obj['Object'] + "\n")
        
# Close file-connection
MyFile.close()

# Close driver
driver.close()

## OpenStreetMap search location example

In [ ]:
# Chrome driver
driver = webdriver.Chrome(service=service, options=options)

driver.get('https://www.openstreetmap.org/#map=9/46.7054/8.0283')
time.sleep(2)

# Submit address
element = driver.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form[1]/div/div[1]/div/input[1]')
element.send_keys("Melchsee-Frutt")
element.send_keys(Keys.RETURN)
time.sleep(2)

# Click info boxes
try:
    # Click 1st info-box
    info = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div[1]/div[5]/div[1]/div/button'))) 
    info.click()

    # Click 2nd info-box
    info = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div[1]/div[6]/div[1]/div/button'))) 
    info.click()

    # Click 3rd info-box
    info = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div[1]/div[7]/button')))
    info.click()
except:
    pass

# Screenshot of map
driver.save_screenshot('screenshot.png')
img = mpimg.imread('screenshot.png')
plt.figure(figsize=(6,6))
plt.axis('off')
imgplot = plt.imshow(img)

# Close driver
driver.close()

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')